In [1]:
pip install requests

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install BeautifulSoup4

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import os
import lxml
from bs4 import BeautifulSoup as bs
from datetime import datetime,timedelta

In [5]:
# 현재 날짜 구하기
now_date = datetime.now()

# 날짜정의
start_date = datetime(2018, 1, 1)
end_date = now_date
cd_min = start_date.strftime('%m/%d/%Y')
cd_max = end_date

# 함수실행
def google_news_data(query, save_path, num_pages=30):
    google_news = []
    
    for page in range(num_pages):
        # Calculate the start parameter for pagination
        start_param = page * 10
        
        # Construct the URL for the current page
        url = f"https://www.google.com/search?q={query}&tbm=nws&tbs=cdr:1,cd_min:{cd_min},cd_max:{cd_max}&start={start_param}"
        
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36'
        }

        response = requests.get(url, headers=headers)
        response.encoding = 'utf-8'
        soup = BeautifulSoup(response.text, 'html.parser')

        for con in soup.select('div.SoaBEf'):
            news_dict = {
                'links': con.find('a')['href'],
                'title': con.select_one('div.MBeuO').get_text(),
                'contents': con.select_one('.GI74Re').get_text(),
                'date': con.select_one('.LfVVr').get_text(),
                'press': con.select_one('.NUnG9d span').get_text(),
            }
            
            # 형식별로 처리
            for i in range(24):  # 최대 24시간까지 처리
                if f'{i}시간 전' in news_dict['date']:
                    news_dict['date'] = now_date.strftime('%Y. %m. %d.')  # 수정
                                          
                elif f'{i}일 전' in news_dict['date']:
                    news_dict['date'] = (now_date - timedelta(days=i)).strftime('%Y. %m. %d.')
                                          
                elif f'{i}주 전' in news_dict['date']:
                    news_dict['date'] = (now_date - timedelta(weeks=i)).strftime('%Y. %m. %d.')
                                          
                elif '1개월 전' in news_dict['date']:
                    news_dict['date'] = (now_date - timedelta(weeks=4)).strftime('%Y. %m. %d.')  # 수정
            
            google_news.append(news_dict)

    df = pd.DataFrame(google_news)
    save_file_path = os.path.join(save_path, "crawling_google_search_SON.csv")
    df.to_csv(save_file_path, index=False, encoding='utf-8-sig')
    print(f"Crawling is complete. The data is saved in {save_file_path}")

    return df

query = input('Enter your search query: ')
save_path = 'C:\\DATA\\DEBA\\SON'
df = google_news_data(query, save_path, num_pages=30)  
print(df.head())


Enter your search query: 세대갈등
Crawling is complete. The data is saved in C:\DATA\DEBA\SON\crawling_google_search_SON.csv
                                               links  \
0  https://www.hani.co.kr/arti/economy/economy_ge...   
1  https://www.donga.com/news/Economy/article/all...   
2    https://www.yna.co.kr/view/AKR20200407134500003   
3  https://www.ntoday.co.kr/news/articleView.html...   
4  http://www.todayenergy.kr/news/articleView.htm...   

                                         title  \
0         팀장은 지도 세대, 막내는 내비게이션 세대…직장내 세대갈등 ...   
1  막내 칼퇴에 팀장이 불편한 이유…상의, ‘직장 내 세대갈등 실태 보고서’ 발간   
2            "막내가 칼퇴?" 직장인 세대차이, 윗세대가 더 많이 느낀다   
3            [청플 Report] 잘못된 MZ 프레이밍이 낳은 세대 갈등   
4                   SK이노베이션, 직장 내 세대 갈등 해결책 마련   

                                            contents          date   press  
0     직장내 세대갈등 보고서. 송채경화 기자. 구독. 등록 2020-04-08 16:29   2020. 4. 8.     한겨레  
1  대한상의, 직장 내 세대갈등으로 본 기업문화 진단 직장인 64% "세대차이 느껴"…...   2020. 4. 8.    동아일보  
2  윗세대로 갈수